In [1]:
import numpy as np 
import pandas as pd 

### `model traning `

In [2]:
df=pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
# df=df.drop(labels=["id"],axis=1) for deleting a column 

In [4]:
del df["id"]

In [5]:
df.head(1)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619


In [6]:
## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [7]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [8]:
Y.head(2)

,price
0,13619
1,13387


In [9]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.columns[X.dtypes=="object"]
numerical_cols = X.columns[X.dtypes!="object"]

In [10]:
categorical_cols


Index(['cut', 'color', 'clarity'], dtype='object')

In [11]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [12]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [13]:
from sklearn.impute import SimpleImputer # handeling  missisg values 
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding

## pipeline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
#numerical pipeline

num_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy='median')),
        ("scaler",StandardScaler())

    ]

)

#catagorical pipeline

cat_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy='most_frequent')),
        ("ordinalencoder",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ("scaler",StandardScaler())
    ]
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [15]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [16]:
y_train

,price
168192,765
35202,4763
41091,6139
31239,720
45722,774
...,...
66455,544
46220,5694
98804,4563
48045,6998


In [17]:
# preprocessor.fit_transform(X_train)

In [18]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [19]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [20]:
# model training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [24]:
regression=LinearRegression()
regression.fit(X_train,y_train)
y_pred_regression=regression.predict(X_test)

In [25]:
print(f"mse=",mean_squared_error(y_test,y_pred_regression))
print(f"mae=",mean_absolute_error(y_test,y_pred_regression))
print(f"rmse=",np.sqrt(mean_squared_error(y_test,y_pred_regression)))
print(f"r2_regression=",r2_score(y_test,y_pred_regression))

mse= 1028002.7598132559
mae= 674.0255115796832
rmse= 1013.9047094344004
r2_regression= 0.9368908248567511


---

In [27]:
lasso=Lasso()
lasso.fit(X_train,y_train)
y_pred_lasso=lasso.predict(X_test)

In [28]:
print(f"mse=",mean_squared_error(y_test,y_pred_lasso))
print(f"mae=",mean_absolute_error(y_test,y_pred_lasso))
print(f"rmse=",np.sqrt(mean_squared_error(y_test,y_pred_lasso)))
print(f"r2_lasso=",r2_score(y_test,y_pred_lasso))

mse= 1027949.4559693959
mae= 675.0716923362162
rmse= 1013.8784226767013
r2_lasso= 0.9368940971841704


In [29]:
ridge=Ridge()
ridge.fit(X_train,y_train)
y_pred_ridge= ridge.predict(X_test)

In [31]:
print(f"mse=",mean_squared_error(y_test,y_pred_ridge))
print(f"mae=",mean_absolute_error(y_test,y_pred_ridge))
print(f"rmse=", np.sqrt(mean_squared_error(y_test,y_pred_ridge)))
print(f"r2_ridge=",r2_score(y_test,y_pred_ridge))

mse= 1028005.2293677655
mae= 674.0555800798253
rmse= 1013.9059272771639
r2_ridge= 0.9368906732505938
